In [2]:

from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import (
    RecursiveCharacterTextSplitter,
)

with open('Example.txt', 'r', encoding='utf-8') as file:
    text = file.read()

text_splitter = RecursiveCharacterTextSplitter(  
    separators = ["\n","\n\n"],    
    chunk_size = 100,
    chunk_overlap  = 50,
    length_function = len,
    is_separator_regex = False,
)
texts = text_splitter.create_documents([text])
texts

[Document(metadata={}, page_content='한국폴리텍대학 학칙 제정 \n1998. 02. 11\n개정 \n2000. 03. 01\n개정 \n2000. 09. 14\n개정 \n2001. 03. 01\n개정 \n2002. 03. 01\n개정'),
 Document(metadata={}, page_content='개정 \n2001. 03. 01\n개정 \n2002. 03. 01\n개정 \n2003. 03. 01\n개정 \n2004. 03. 01\n개정 \n2005. 03. 01\n개정'),
 Document(metadata={}, page_content='개정 \n2004. 03. 01\n개정 \n2005. 03. 01\n개정 \n2005. 04. 21\n개정 \n2006. 03. 01\n개정 \n2007. 03. 01\n개정'),
 Document(metadata={}, page_content='개정 \n2006. 03. 01\n개정 \n2007. 03. 01\n개정 \n2007. 08. 20\n개정 \n2008. 03. 01\n개정 \n2009. 03. 01\n개정'),
 Document(metadata={}, page_content='개정 \n2008. 03. 01\n개정 \n2009. 03. 01\n개정 \n2010. 01. 01\n개정 \n2010. 03. 01\n개정 \n2010. 09. 01\n개정'),
 Document(metadata={}, page_content='개정 \n2010. 03. 01\n개정 \n2010. 09. 01\n개정 \n2011. 03. 01\n개정 \n2012. 03. 01\n개정 \n2012. 07. 03\n개정'),
 Document(metadata={}, page_content='개정 \n2012. 03. 01\n개정 \n2012. 07. 03\n개정 \n2013. 03. 01\n개정 \n2014. 03. 01\n개정 \n2015. 03. 01\n개정'),
 Document(metadata={}, 

In [4]:
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
embedding_function = SentenceTransformerEmbeddings(model_name="jhgan/ko-sroberta-multitask")

In [6]:
from langchain.vectorstores import FAISS

# Create FAISS vector store from the documents and their embeddings
db = FAISS.from_documents(texts, embedding_function)

# Verify the store by performing a quick similarity search
query = "졸업하려면 알아야하는 학칙이 뭐야."
docs = db.similarity_search(query)
print(docs[0].page_content)  # Print the first retrieved document



제2조(경과조치) 이 학칙 시행당시 시각디자인과, 컴퓨터애니메이션과에 재적중인 학생은 커뮤니케이션디자인과에, 멀티미디어과, 컴퓨터게임과에 재적중인 학생은 미디어콘텐츠과에, 패션디자인과에 재적중인 학생은 패션메이킹과에 재적하고 있는 것으로 본다. 부     칙


In [7]:
# Create a retriever for retrieving relevant documents
retriever = db.as_retriever(search_type="similarity", search_kwargs={'k': 5, 'fetch_k': 50})

# Test the retriever with a query
retrieved_docs = retriever.get_relevant_documents("졸업하려면 알아야하는 학칙.")
print(retrieved_docs[0].page_content)  # Print the first relevant document



제25조(자퇴) 질병, 기타 부득이한 사유로 자퇴하고자 하는 자는 보호자 연서로 사유를 명시한 자퇴원을 제출하여 학장(지역대학장)의 허가를 받아야 한다. 단, 신청일 기준 미성년자일 경우 보호자 연서로 사유서를 제출하여야 한다.<개정 `18.2.6, 20.3.1,  21.3.1>


C:\Users\AISW-203-122\AppData\Local\Temp\ipykernel_24088\312397901.py:5: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retrieved_docs = retriever.get_relevant_documents("상속이 뭐야.")


In [8]:
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnableMap

template = """
너는 학칙 전문가야. 반드시 모든 대답은 한글로 해주세요.
제공하는 맥락만을 사용하여 사용자의 질문에 답해주세요.
맥락에 나타나지 않은 정보는 알지 못 한다고 안내해야만 합니다.

맥락:
{context}

사용자의 질문: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [10]:
from langchain_community.chat_models import ChatOllama
llm = ChatOllama(model="gemma2:9b", temperature=0, base_url="http://117.111.249.205:51434/") #http://127.0.0.1:11434


In [14]:
from langchain.schema.runnable import RunnableMap

# Create a chain that retrieves relevant documents and passes them to the LLM
chain = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x['question']),
    "question": lambda x: x['question']
}) | prompt | llm

# Test the chain with a question
result = chain.invoke({'question': "졸업하려면 알아야하는 학칙이 뭐야"})
print(result.content)


2020학년도 입학생부터는 계약학과에서 졸업하기 위해서는 교양교과(6학점 이상)를 포함하여 80학점 이상을 이수해야 합니다.  





In [13]:
from IPython.display import Markdown
Markdown(chain.invoke({'question': "학칙 뭐야"}).content)

제공된 문서에는 한국폴리텍대학 학칙에 대한 내용이 담겨 있습니다.  




In [40]:
import requests
key = 'I3imu+UUWxMLVLf6ImvuPB71ZXzCdIW0KAHyuRDKDtOYGK/HWZdE6TZvPjCbj3NlV2KTxUKRKGESkpXbeBSWJw=='
url = 'http://apis.data.go.kr/B552584/ArpltnInforInqireSvc/getCtprvnRltmMesureDnsty'
params ={'serviceKey' : key, 'returnType' : 'json', 'numOfRows' : '100', 'pageNo' : '1', 'sidoName' : '서울', 'ver' : '1.0' }

response = requests.get(url, params=params)
print(response.content)

b'{"response":{"body":{"totalCount":40,"items":[{"so2Grade":"1","coFlag":null,"khaiValue":"92","so2Value":"0.003","coValue":"0.3","pm25Flag":null,"pm10Flag":null,"o3Grade":"2","pm10Value":"23","khaiGrade":"2","pm25Value":"18","sidoName":"\xec\x84\x9c\xec\x9a\xb8","no2Flag":null,"no2Grade":"1","o3Flag":null,"pm25Grade":"1","so2Flag":null,"dataTime":"2024-09-26 15:00","coGrade":"1","no2Value":"0.009","stationName":"\xec\xa4\x91\xea\xb5\xac","pm10Grade":"1","o3Value":"0.081"},{"so2Grade":"1","coFlag":null,"khaiValue":"93","so2Value":"0.003","coValue":"0.5","pm25Flag":null,"pm10Flag":null,"o3Grade":"2","pm10Value":"41","khaiGrade":"2","pm25Value":"18","sidoName":"\xec\x84\x9c\xec\x9a\xb8","no2Flag":null,"no2Grade":"1","o3Flag":null,"pm25Grade":"2","so2Flag":null,"dataTime":"2024-09-26 15:00","coGrade":"1","no2Value":"0.026","stationName":"\xed\x95\x9c\xea\xb0\x95\xeb\x8c\x80\xeb\xa1\x9c","pm10Grade":"2","o3Value":"0.082"},{"so2Grade":"1","coFlag":null,"khaiValue":"94","so2Value":"0.003","c

In [41]:
import json
import requests
x = "서울"

def seoul_pm_query(sido ,key):
    url = 'http://apis.data.go.kr/B552584/ArpltnInforInqireSvc/getCtprvnRltmMesureDnsty'
    params ={'serviceKey' : key, 'returnType' : 'json', 'numOfRows' : '100', 'pageNo' : '1', 'sidoName' : sido, 'ver' : '1.0' }    
    response = requests.get(url, params=params)
    content = response.content.decode('utf-8')
    data = json.loads(content)
    print(data)
    return data

y = seoul_pm_query(x,key)


{'response': {'body': {'totalCount': 40, 'items': [{'so2Grade': '1', 'coFlag': None, 'khaiValue': '92', 'so2Value': '0.003', 'coValue': '0.3', 'pm25Flag': None, 'pm10Flag': None, 'o3Grade': '2', 'pm10Value': '23', 'khaiGrade': '2', 'pm25Value': '18', 'sidoName': '서울', 'no2Flag': None, 'no2Grade': '1', 'o3Flag': None, 'pm25Grade': '1', 'so2Flag': None, 'dataTime': '2024-09-26 15:00', 'coGrade': '1', 'no2Value': '0.009', 'stationName': '중구', 'pm10Grade': '1', 'o3Value': '0.081'}, {'so2Grade': '1', 'coFlag': None, 'khaiValue': '93', 'so2Value': '0.003', 'coValue': '0.5', 'pm25Flag': None, 'pm10Flag': None, 'o3Grade': '2', 'pm10Value': '41', 'khaiGrade': '2', 'pm25Value': '18', 'sidoName': '서울', 'no2Flag': None, 'no2Grade': '1', 'o3Flag': None, 'pm25Grade': '2', 'so2Flag': None, 'dataTime': '2024-09-26 15:00', 'coGrade': '1', 'no2Value': '0.026', 'stationName': '한강대로', 'pm10Grade': '2', 'o3Value': '0.082'}, {'so2Grade': '1', 'coFlag': None, 'khaiValue': '94', 'so2Value': '0.003', 'coValue'

In [45]:
def parse_air_quality_data(data):
    items = data['response']['body']['items']
    
    air_quality_info = []
    
    for item in items:
        info = {
            '측정소명': item.get('stationName'),
            '날짜': item.get('dataTime'),
            'pm10농도': item.get('pm10Value'),
            'pm25농도': item.get('pm25Value'),
            'so2농도': item.get('so2Value'),
            'co농도': item.get('coValue'),
            'o3농도': item.get('o3Value'),
            'no2농도': item.get('no2Value'),
            '통합대기환경수치': item.get('khaiValue'),
            '통합대기환경지수': item.get('khaiGrade'),
            'pm10등급': item.get('pm10Grade'),
            'pm25등급': item.get('pm25Grade')
        }
        air_quality_info.append(info)
    return air_quality_info

air_quality_info = parse_air_quality_data(0)
air_quality_info[0]


{'측정소명': '중구',
 '날짜': '2024-09-26 15:00',
 'pm10농도': '23',
 'pm25농도': '18',
 'so2농도': '0.003',
 'co농도': '0.3',
 'o3농도': '0.081',
 'no2농도': '0.009',
 '통합대기환경수치': '92',
 '통합대기환경지수': '2',
 'pm10등급': '1',
 'pm25등급': '1'}

In [49]:
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.schema import Document

documents = [Document(page_content=", ".join([f"{key}: {str(info[key])}" for key in ['측정소명', '날짜', 'pm10농도', 'pm25농도', '통합대기환경수치']])) for info in air_quality_info]
print(documents)

[Document(metadata={}, page_content='측정소명: 중구, 날짜: 2024-09-26 15:00, pm10농도: 23, pm25농도: 18, 통합대기환경수치: 92'), Document(metadata={}, page_content='측정소명: 한강대로, 날짜: 2024-09-26 15:00, pm10농도: 41, pm25농도: 18, 통합대기환경수치: 93'), Document(metadata={}, page_content='측정소명: 종로구, 날짜: 2024-09-26 15:00, pm10농도: 24, pm25농도: 16, 통합대기환경수치: 94'), Document(metadata={}, page_content='측정소명: 청계천로, 날짜: 2024-09-26 15:00, pm10농도: 30, pm25농도: 19, 통합대기환경수치: -'), Document(metadata={}, page_content='측정소명: 종로, 날짜: 2024-09-26 15:00, pm10농도: 32, pm25농도: 16, 통합대기환경수치: 81'), Document(metadata={}, page_content='측정소명: 용산구, 날짜: 2024-09-26 15:00, pm10농도: 28, pm25농도: 18, 통합대기환경수치: 85'), Document(metadata={}, page_content='측정소명: 광진구, 날짜: 2024-09-26 15:00, pm10농도: 27, pm25농도: 20, 통합대기환경수치: 115'), Document(metadata={}, page_content='측정소명: 성동구, 날짜: 2024-09-26 15:00, pm10농도: 30, pm25농도: 17, 통합대기환경수치: 93'), Document(metadata={}, page_content='측정소명: 강변북로, 날짜: 2024-09-26 15:00, pm10농도: 33, pm25농도: -, 통합대기환경수치: -'), Document(metadata={

In [54]:
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnableMap

template = """
너는 미세먼지 환경 전문가야. 반드시 모든 대답은 한글로 해주세요.
제공하는 맥락만을 사용하여 사용자의 질문에 답해주세요.
맥락에 나타나지 않은 정보는 알지 못 한다고 안내해야만 합니다.

맥락:
{context}

사용자의 질문: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [82]:
from sentence_transformers import SentenceTransformer, util
import numpy as np

embedder = SentenceTransformer("jhgan/ko-sroberta-multitask")

# Corpus with example sentences
corpus = ['용산구 미세먼지:',
        '이산화 탄소 수치',
        '용산구 온도',]

corpus_embeddings = embedder.encode(corpus, convert_to_tensor=True)
print(corpus_embeddings)



c:\Users\AISW-203-122\Desktop\HoProject1\testpy310\lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


tensor([[-0.5708, -0.3057,  0.4384,  ..., -0.0035,  0.0660,  0.3504],
        [ 0.1370, -0.7294,  0.9680,  ...,  0.1748, -0.0661, -0.1366],
        [-0.5416, -0.1987,  0.4724,  ..., -0.6458,  0.3659,  0.2952]])


In [76]:
from langchain.vectorstores import FAISS
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain_community.chat_models import ChatOllama
from langchain.schema.runnable import RunnableMap


documents_as_texts = [str(item) for item in air_quality_info]


embedder = SentenceTransformerEmbeddings(model_name="jhgan/ko-sroberta-multitask")


db = FAISS.from_texts(documents_as_texts, embedder)



c:\Users\AISW-203-122\Desktop\HoProject1\testpy310\lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [81]:

query = "용산구 대기환경"
docs = db.similarity_search(query)


relevant_data = "\n".join([doc.page_content for doc in docs])


llm = ChatOllama(model="gemma2:9b", temperature=0, base_url="http://117.111.249.205:51434/")

question = "용산구 대기환경을 수치로 보여줘"


template = """
환경에 관한 수치를 분석하고 그 분석한 결과를 통해서 
그 날 하루 놀러갈만한 곳을 추천해줘
{context}

질문: {question}


"""


prompt = ChatPromptTemplate.from_template(template)


chain = RunnableMap({
    "context": lambda x: relevant_data,  # 검색된 관련 데이터를 문맥으로 전달
    "question": lambda x: x['question']  # 사용자의 질문
}) | prompt | llm


response = chain.invoke({'question': question})
print(response.content)

용산구의 대기 환경은 다음과 같습니다. 

* **날짜:** 2024-09-26 15:00
* **pm10농도:** 28
* **pm25농도:** 18
* **so2농도:** 0.003
* **co농도:** 0.4
* **o3농도:** 0.072
* **no2농도:** 0.024
* **통합대기환경수치:** 85
* **통합대기환경지수:** 2
* **pm10등급:** 1
* **pm25등급:** 2



